# 📘 Limpieza Instagram 

## 🧰 Librerías e importaciones
- Importa: pandas, re.

In [1]:
import re
import json
import emoji
import pandas as pd
from langdetect import detect, DetectorFactory

In [3]:
DetectorFactory.seed = 0

## 🧼 Limpieza de texto
- Quita URLs del texto.
- Gestiona menciones/hashtags (#, @).

In [5]:
def limpiar_texto(texto):
    texto = texto.lower()
    texto = emoji.demojize(texto, language="es")
    texto = texto.replace(":", "").replace("_", " ")
    texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)  # URLs
    texto = re.sub(r"[@#]", "", texto)  # elimina solo @ y #
    texto = re.sub(r"\S+@\S+\.\S+", "", texto)  # correos
    texto = re.sub(r"\(.*?\)|\[.*?\]|\{.*?\}", "", texto)  # paréntesis
    texto = re.sub(r"\d+", "", texto)  # números
    texto = re.sub(r"[^a-záéíóúüñ\s]", "", texto)  # caracteres raros
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

## 📥 Carga de datos
- Lee dataset: Tenerife_instagram_posts_20250829.json.

In [7]:
with open("Tenerife_instagram_posts_20250829.json", "r", encoding="utf-8") as f:
    data = json.load(f)

## 🧪 Bloque de código
- Operaciones varias sobre el DataFrame, creacción de diccionario.

In [9]:
keywords = {
    "es": {
        "hotel": ["hotel", "hostal"],
        "playa": ["playa"],
        "tour": ["tour", "excursión"],
        "deporte": ["deporte", "deportes"],
        "cultura": ["cultura"],
        "comida": ["comida", "restaurante", "platos", "platillos", "comer"],
        "museo": ["museo"],
        "atracciones": ["atracciones", "actividades"]
    },
    "en": {
        "hotel": ["hotel", "hostel"],
        "playa": ["beach"],
        "tour": ["tour", "excursion"],
        "deporte": ["sports", "sport"],
        "cultura": ["culture"],
        "comida": ["food", "restaurant", "dishes", "saucers", "eat"],
        "museo": ["museum"],
        "attractions": ["attractions", "activities"]
    },
    "fr": {
        "hotel": ["hôtel", "auberge"],
        "playa": ["plage"],
        "tour": ["visite", "excursion"],
        "deporte": ["sport", "sports"],
        "cultura": ["culture", "culturel"],
        "comida": ["nourriture", "repas", "restaurant", "plats", "soucoupes", "manger"],
        "museo": ["musée"],
        "attractions": ["attractions", "activités" ]
    },
    "it": {
        "hotel": ["hotel", "ostello"],
        "playa": ["spiaggia"],
        "tour": ["tour", "escursione", "giro"],
        "deporte": ["sport", "attività sportiva"],
        "cultura": ["cultura", "culturale"],
        "comida": ["cibo", "ristorante", "pasto", "piatti", "piattini", "mangiare"],
        "museo": ["museo", "galleria"],
        "attrazioni": ["attrazioni", "attività"]
    }
}

## 🛠️ Helpers / funciones auxiliares
- Define funciones reutilizables para la limpieza.

In [11]:
def asignar_categoria(texto):
    try:
        idioma = detect(texto)
    except:
        idioma = "unknown"
    
    if idioma not in keywords:
        return "Desconocido", idioma
    
    for categoria, palabras in keywords[idioma].items():
        for palabra in palabras:
            if palabra in texto:
                return categoria.capitalize(), idioma
    return "Desconocido", idioma

In [13]:
rows = []
for post in data:
    comentario = ""
    if "titulo" in post and post["titulo"]:
        comentario += post["titulo"] + " "
    if "texto" in post and post["texto"]:
        comentario += post["texto"]

    comentario_limpio = limpiar_texto(comentario)
    categoria, idioma = asignar_categoria(comentario_limpio)

    # Armar fila
    rows.append({
        "texto": comentario_limpio,
        "ciudad": post.get("ciudad", ""),
        "experiencia": categoria,   # dejar en blanco
        "fecha": post.get("fecha", ""),
        "fuente": "Instagram"
    })

In [14]:
df = pd.DataFrame(rows)

## ⏱️ Parseo/normalización de fechas
- Convierte columnas de fecha a formato datetime y ajusta formato.

In [15]:
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce").dt.date

## 🧪 Muestra DF

In [16]:
df[df['experiencia'] == "Desconocido"]

,texto,ciudad,experiencia,fecha,fuente
0,loroparque tenerife tenerifeturismo delfines,Tenerife,Desconocido,2022-05-02,Instagram
2,el casco histórico de la orotava conserva su e...,Tenerife,Desconocido,2021-05-02,Instagram
4,cuando los sentimientos esos que nos hacen vul...,Tenerife,Desconocido,2022-04-04,Instagram
6,nuestros antiguos aborígenes corazón negro can...,Tenerife,Desconocido,2020-05-04,Instagram
7,taller de iniciación al salto del pastor para ...,Tenerife,Desconocido,2021-09-13,Instagram
...,...,...,...,...,...
412,la caleta a beautiful spot in tenerifechispas ...,Tenerife,Desconocido,2021-05-28,Instagram
413,clásicos atemporales corazón negrocorazón negr...,Tenerife,Desconocido,2021-06-18,Instagram
414,dedo índice hacia arribacara guiñando el ojo g...,Tenerife,Desconocido,2023-04-02,Instagram
416,when he makes you mad whattodo lifehacks teide...,Tenerife,Desconocido,2018-11-14,Instagram


## 💾 Guardado de resultados
- Exporta a: Tenerife_limpios.csv.

In [21]:
df.to_csv("Tenerife_limpios.csv", index=False, encoding="utf-8")